<a href="https://colab.research.google.com/github/ErikSeras/usos_gee/blob/master/analisis_atmosfera/gif_goes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Cómo poder visualizar un huracán?

Los huracanes son las tormentas más grandes y violentas de la Tierra. Las personas llaman a estas tormentas con distintos nombres como tifones o ciclones según el lugar donde se producen. El término científico para todas estas tormentas es ciclón tropical. Sólo los ciclones tropicales que se forman sobre el Océano Atlántico y el Océano Pacífico oriental se llaman "huracanes" [(NASA, 2019)](https://spaceplace.nasa.gov/hurricanes/sp/).

El seguimiento de estos es crucial para la toma de acciones de los países que puedan ser afectados por su paso.

Hay diferentes formas de visualizar estos eventos desde el espacio con satélites, una de ellas es mediante el uso de las imágenes captadas por el Satélite Geoestacionario Ambiental Operacional - R Serie [(GOES-R)](https://www.goes-r.gov/), antes conocido como GOES 16. La plataforma [GOES Image Viewer](https://www.star.nesdis.noaa.gov/GOES/fulldisk.php?sat=G16) permite visualizar las imágenes mediante la combinación de diferentes bandas. Pero para este ejemplo, usaremos la plataforma de Google Earth Engine (GEE), el cual permite manejar todo el dataset de imágenes y poder visualizar eventos, como el huracán Genenieve durante agosto del 2020.






## Generar un GIF que represente el movimiento de un huracán

El siguiente código escrito en JavaScript permite generar un GIF que representa el huracán Genenieve durante el 18 de agosto del 2020. Este código funciona en la plataforma de Google Earth Engine. Este [enlace](https://code.earthengine.google.com/3935d5a7d848084b99fb91a44a154184) redirecciona al script en la plataforma de GEE.

```{js}
////////////////////////////////////////////////////////////////////////////////

// Elaborado por: Erik Seras
// Fecha: 2020-09-30

// Objetivo del script
// Visualizar el huracán Genevieve durante el 2020-08-18

////////////////////////////////////////////////////////////////////////////////

// 1. Definir la región de estudio

/*
Elección de la región adecuada para poder visualizar del paso
del huracán Genevieve. En caso desea analizar otro evento
parecido, traze un rectángulo o la forma que desee. Una
vez satisfecho con la área definida, puede dejarlo como
una variable importada, o escribirla mediante el objeto
ee.Geometry.Polygon().
*/

// Área del estudio
var roi = ee.Geometry.Polygon(
  [[
    [-118, 10],
    [-98,10],
    [-98,30],
    [-118,30],
    [-118,10]]], null,
  false);

////////////////////////////////////////////////////////////////////////////////

// 2. Arreglo de la data

/*
Earth Engine Data Catalog brinda información sobre los
diferentes dataset disponibles. En la descripción del
dataset usada en este ejercicio NOAA/GOES/16/MCMIPF brinda
una script que permite corregir los datos de las imágenes
y facilitar su rápido análisis.

Escribir NOAA/GOES/16/MCMIPF en el buscador de
https://developers.google.com/earth-engine/datasets para
conocer más sobre este dataset.
*/

// Función que arregla los valores de las imágenes

// Consideraciones para la función
// Alias de las bandas.
var BLUE = 'CMI_C01';
var RED = 'CMI_C02';
var VEGGIE = 'CMI_C03';
var GREEN = 'GREEN';
// Número de bandas
var NUM_BANDS = 33;
// Omitir las bandas DQF intercaladas.
var BLUE_BAND_INDEX = (1 - 1) * 2;
var RED_BAND_INDEX = (2 - 1) * 2;
var VEGGIE_BAND_INDEX = (3 - 1) * 2;
var GREEN_BAND_INDEX = NUM_BANDS - 1;

// Rango de visualización para GOES RGB
var GOES_MIN = 0.0;
var GOES_MAX = 1;  // Alternativamente 1.3.

// Crear la función

/*
Esta función sirve para arreglar todas las imágenes de un dataset
usando la función map (). Pero también sirve para arreglar una sola.
*/

var applyScaleAndOffset = function(image) {
  image = ee.Image(image);
  var bands = new Array(NUM_BANDS);
  for (var i = 1; i < 17; i++) {
    var bandName = 'CMI_C' + (100 + i + '').slice(-2);
    var offset = ee.Number(image.get(bandName + '_offset'));
    var scale =  ee.Number(image.get(bandName + '_scale'));
    bands[(i-1) * 2] = image.select(bandName).multiply(scale).add(offset);

    var dqfName = 'DQF_C' + (100 + i + '').slice(-2);
    bands[(i-1) * 2 + 1] = image.select(dqfName);
  }
  
  /* Estudio sobre generar imagenes de color verdadero con GOES-16
  
  Bah, Gunshor, Schmit, Generation of GOES-16 True Color Imagery without a
  Green Band, 2018. https://doi.org/10.1029/2018EA000379
  */
  // Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
  var green1 = bands[RED_BAND_INDEX].multiply(0.45);
  var green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10);
  var green3 = bands[BLUE_BAND_INDEX].multiply(0.45);
  var green = green1.add(green2).add(green3);
  bands[GREEN_BAND_INDEX] = green.rename(GREEN);

  return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()));
};

////////////////////////////////////////////////////////////////////////////////

// 3. Generación del GIF

// Filtrar por fechas el dataset
// La fecha 2020-08-18T15:00:00 tiene como zona horaria UTC por defecto.

var goes16 = ee.ImageCollection("NOAA/GOES/16/MCMIPF")
              .filterDate("2020-08-18T15:00:00", "2020-08-18T23:00:00");

// Aplicar la correción
var goes16_arreglado = goes16.map(applyScaleAndOffset);

// Función pra cortar las imágenes en un Image Collection
var cortarRoi = function(image){
  return image.clip(roi);
};

// Cortar las imágenes por la región de interés
var goes16_roi = goes16_arreglado.map(cortarRoi);

// Características del GIF a generar
var videoArgs = {
  min: 0,
  max: 1,
  dimensions: 600,
  region: roi,
  framesPerSecond: 12,
  crs: 'EPSG:3857',
  bands: ['CMI_C02', 'GREEN', 'CMI_C01']
};

// Mostrar el GIF en consola
print(ui.Thumbnail(goes16_roi, videoArgs));

////////////////////////////////////////////////////////////////////////////////
```





El GIF generado en la plataforma debe ser descargado a su computadora. Luego puede guardarla en algún lugar de su Google Drive y luego concectarlo con Google Colab, o subir de manera directa el GIF mediante la carga de archivos de Google Colab.

## Añadir texto e imágenes al GIF

El paquete de Python [`geemap`](https://github.com/giswqs/geemap) permite el mapeo interactivo con GEE. Para esta ocasión usaremos las funciones que permiten editar GIF generados con GEE.


Deberá instalar el paquete cada vez que use el archivo de jupyter notebook, al igual que iniciar sesión en GEE para poder usar las funcionalidades y los dataset.

In [1]:
# Función mágica que suprime los resultados
%%capture
## Instalar geemap
!pip install geemap

In [2]:
## Librerías necesarias
import ee
import geemap.eefolium as geemap

In [ ]:
# Iniciar sesión de GEE
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Si guardó el GIF generado con anteriodad en Google Drive y pudo importarlo a Google Colab o lo subió directamente a la sesión, ya no es necesario correr el siguiente código. `! wget 'enlace del archivo'` permite importar directamente archivos desde repositorios de github o enlaces de páginas web.

In [4]:
%%capture

# Importar el gif desde GitHub
! wget https://github.com/ErikSeras/usos_gee/blob/master/analisis_atmosfera/data/huracan_genevieve_2020_08_18.gif?raw=true

GIF importado desde GitHub para el desarrollo del ejemplo

![Ejemplo de GIF generado con GEE](https://github.com/ErikSeras/usos_gee/blob/master/analisis_atmosfera/data/huracan_genevieve_2020_08_18.gif?raw=true)

La siguiente celda permite cambiar el nombre del archivo, ya que al importar directamente puede ser que tenga algun texto adicional. En este ejemplo se usó `! mv` para poder cambiar de `huracan_genevieve_2020_08_18.gif?raw=true` a `huracan_genevieve_2020_08_18.gif`. Si importó de Google Drive o directamente el GIF ya no es necesario ejecutarlo.

In [5]:
# Arreglar el nombre y dejarlo con la terminación .gif
! mv huracan_genevieve_2020_08_18.gif?raw=true huracan_genevieve_2020_08_18.gif

In [6]:
# Función que permite visualizar el GIF
geemap.show_image('/content/huracan_genevieve_2020_08_18.gif')

Output()

Es necesario crear direcciones de archivos que permitan acceder al GIF inicial y al GIF modificado ya que será utilizado para poder ejecutar las funciones de adición de texto e imágenes.

In [7]:
## Dirección del GIF a modificar
in_gif = '/content/huracan_genevieve_2020_08_18.gif'

## Dirección del GIF modificado
out_gif = '/content/huracan_genevieve_final.gif'

Deberá tener en cuenta que cosas quiere añadir al GIF. Por ejemplo poner un título, el autor y la fecha de análisis, o inclusive alguna imagen que represente la fuente de los datos analizados. La siguiente celda contiene algunos argumentos que requiere las funciones `geemap.add_text_to_gif()` y `geemap.add_image_to_giff()` que permiten personalizar el GIF final.

In [8]:
# Argumentos para geemap.add_text_to_gif()

# Título general del GIF
titulo = "GOES-16 MCMIPF Series ABI Level 2\nCloud and Moisture Imagery Full Disk\n                Huracán Genevieve"

# Nombre del autor del GIF
autor = "Procesado por: Erik Seras" # Poner su nombre

# Color del texto
color_texto = '#9ef0ff' # Formato hex

# Tamaño de la fuente
tamano_texto = 15

# Fecha
fecha = 'Fecha: 2020-08-18'

# En el ejemplo se tiene un gif con 12 fps, para mantenerlo se debe
# arreglar la duración en base a lo predeterminado de la función
fps_original = 12 # fps
mantener_fps = 100/fps_original*10 # 83.33 milisegundos por cada fotograma

# Aunque puede poner un valor mas redondo a la decena manualmente, como 80.

#  Argumentos para geemap.add_image_to_giff()

# Añadir algunos logos al gif
noaa_logo = 'https://bit.ly/3ahJoMq'
ee_logo = 'https://bit.ly/2RJYSCo'

### Añadir texto al GIF

In [9]:
# Función para añadir texto a un GIF
geemap.add_text_to_gif(
    # Dirección del gif inicial y saliente
    in_gif, out_gif,
    # Texto que se desea agregar al GIF
    text_sequence = titulo,
    # Ubicación del texto en el GIF
    xy = ('30%', '5%'),
    # Color y tamaño del texto
    font_color = color_texto, font_size = tamano_texto,
    # Velocidad de cada fotograma en milisegundos
    duration = mantener_fps
)

In [10]:
# Función para añadir texto a un GIF
geemap.add_text_to_gif(
    # Dirección del gif inicial y saliente
    out_gif, out_gif,
    # Ubicación del texto en el GIF
    xy=('65%', '95%'),
    text_sequence = autor,
    # Color y tamaño del texto
    font_color = color_texto, font_size = tamano_texto,
    # Velocidad de cada fotograma en milisegundos
    duration = mantener_fps
)

In [11]:
# Función para añadir texto a un GIF
geemap.add_text_to_gif(
    # Dirección del gif inicial y saliente
    out_gif, out_gif,
    # Ubicación del texto en el GIF
    xy=('5%', '95%'),
    text_sequence = fecha,
    # Color y tamaño del texto
    font_color = color_texto, font_size = tamano_texto,
    # Velocidad de cada fotograma en milisegundos
    duration = mantener_fps
)

### Añadir imágenes al GIF

In [12]:
# Añadir imagen al GIF
geemap.add_image_to_gif(out_gif, out_gif, in_image=noaa_logo, xy = ('6%', '85%'), image_size=(60, 60))

In [13]:
# Añadir imagen al GIF
geemap.add_image_to_gif(out_gif, out_gif, in_image=ee_logo, xy = ('16%', '85%'), image_size=(60, 60))

In [15]:
# Ver el GIF final
geemap.show_image(out_gif)

Output()